# Simple strategy 30 min verbeteringen?

Dit notebook probeert de huidige simple strat 30 min te verbeteren.
Beoogde verbeteringen zijn:

* Initiele stoploss van -20% aanpassen naar een vorm van trailing stoploss
* exit point (nu op basis van crossdown macd signal line) verbeteren

In [27]:
# Importeer de benodigde bibliotheken
import pandas as pd
from freqtrade.strategy.interface import IStrategy
from pandas import DataFrame
import talib.abstract as ta
import freqtrade.vendor.qtpylib.indicators as qtpylib
from scipy.spatial.distance import cosine
import numpy as np

In [28]:
# Load data
df = pd.read_json("BTC_USDT-30m.json")

In [29]:
df.head()

,0,1,2,3,4,5
0,1502942400000,4261.48,4280.56,4261.32,4261.45,11.308926
1,1502944200000,4280.00,4313.62,4267.99,4308.83,35.872083
2,1502946000000,4308.83,4328.69,4304.31,4320.00,21.048648
3,1502947800000,4320.00,4320.00,4291.37,4315.32,2.186268
4,1502949600000,4330.29,4330.29,4309.37,4311.02,3.566277


In [30]:
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

,date,open,high,low,close,volume
65265,2021-05-13 06:30:00,50845.04,51335.00,50760.42,51332.74,2062.857647
65266,2021-05-13 07:00:00,51332.74,51367.19,50943.64,50947.68,1827.847841
65267,2021-05-13 07:30:00,50947.67,51000.00,50605.38,50879.72,1804.209253
65268,2021-05-13 08:00:00,50879.73,50950.00,50205.35,50476.23,2284.016803
65269,2021-05-13 08:30:00,50478.70,50674.99,49938.44,50015.82,2332.628022


In [31]:
df.shape

(65270, 6)

In [32]:
df.info

<bound method DataFrame.info of                      date      open      high       low     close       volume
0     2017-08-17 04:00:00   4261.48   4280.56   4261.32   4261.45    11.308926
1     2017-08-17 04:30:00   4280.00   4313.62   4267.99   4308.83    35.872083
2     2017-08-17 05:00:00   4308.83   4328.69   4304.31   4320.00    21.048648
3     2017-08-17 05:30:00   4320.00   4320.00   4291.37   4315.32     2.186268
4     2017-08-17 06:00:00   4330.29   4330.29   4309.37   4311.02     3.566277
...                   ...       ...       ...       ...       ...          ...
65265 2021-05-13 06:30:00  50845.04  51335.00  50760.42  51332.74  2062.857647
65266 2021-05-13 07:00:00  51332.74  51367.19  50943.64  50947.68  1827.847841
65267 2021-05-13 07:30:00  50947.67  51000.00  50605.38  50879.72  1804.209253
65268 2021-05-13 08:00:00  50879.73  50950.00  50205.35  50476.23  2284.016803
65269 2021-05-13 08:30:00  50478.70  50674.99  49938.44  50015.82  2332.628022

[65270 rows x 6 col

## Indicators toevoegen



In [46]:
df["rsi"] = ta.RSI(df, timeperiod=14)
df["sma"] = ta.SMA(df, timeperiod=23)
stoch = ta.STOCH(
    df,
    fastk_period=14,
    slowk_period=4,
    slowk_matype=0,
    slowd_period=6,
    slowd_matype=0,
)
df['slowd'] = stoch['slowd']
df['slowk'] = stoch['slowk']

In [47]:
df.tail(10)

,date,open,high,low,close,volume,rsi,sma,slowd,slowk,rsi_sma,rsi_sma_cross,stoch_sell_cross
65260,2021-05-13 04:00:00,50461.99,50986.22,50380.30,50921.01,2609.544416,37.472958,52329.414348,48.290663,49.826036,38.951416,False,False
65261,2021-05-13 04:30:00,50921.01,50921.01,50282.00,50445.87,2137.740682,34.641282,52129.455652,49.116637,50.376265,38.281345,False,False
65262,2021-05-13 05:00:00,50445.87,51076.41,50444.00,51051.72,1922.728800,40.785695,51963.655217,50.747652,57.601310,37.769987,True,False
65263,2021-05-13 05:30:00,51051.73,51063.46,50799.90,50884.09,1834.320099,39.674328,51812.129565,52.826876,62.422843,37.375690,False,False
65264,2021-05-13 06:00:00,50884.09,51247.05,50813.05,50842.29,1925.525253,39.386122,51671.250000,56.118426,68.203153,36.960267,False,False
65265,2021-05-13 06:30:00,50845.04,51335.00,50760.42,51332.74,2062.857647,44.482115,51550.325652,61.485844,80.485459,36.722699,False,False
65266,2021-05-13 07:00:00,51332.74,51367.19,50943.64,50947.68,1827.847841,41.529976,51398.816522,66.900729,82.315344,36.476152,False,False
65267,2021-05-13 07:30:00,50947.67,51000.00,50605.38,50879.72,1804.209253,41.012633,51262.160435,72.477554,83.837218,36.256573,False,False
65268,2021-05-13 08:00:00,50879.73,50950.00,50205.35,50476.23,2284.016803,37.987001,51090.686087,75.673123,76.774719,35.907044,False,False
65269,2021-05-13 08:30:00,50478.70,50674.99,49938.44,50015.82,2332.628022,34.829508,50886.293043,74.978484,58.255013,35.538514,False,False


## Berekeningen op indicators

In [39]:
df["rsi_sma"] = df["rsi"].rolling(window=50).mean()
df.tail(10)

,date,open,high,low,close,volume,rsi,sma,slowd,slowk,rsi_sma,rsi_sma_cross
65260,2021-05-13 04:00:00,50461.99,50986.22,50380.30,50921.01,2609.544416,37.472958,52329.414348,48.290663,49.826036,38.951416,False
65261,2021-05-13 04:30:00,50921.01,50921.01,50282.00,50445.87,2137.740682,34.641282,52129.455652,49.116637,50.376265,38.281345,False
65262,2021-05-13 05:00:00,50445.87,51076.41,50444.00,51051.72,1922.728800,40.785695,51963.655217,50.747652,57.601310,37.769987,True
65263,2021-05-13 05:30:00,51051.73,51063.46,50799.90,50884.09,1834.320099,39.674328,51812.129565,52.826876,62.422843,37.375690,False
65264,2021-05-13 06:00:00,50884.09,51247.05,50813.05,50842.29,1925.525253,39.386122,51671.250000,56.118426,68.203153,36.960267,False
65265,2021-05-13 06:30:00,50845.04,51335.00,50760.42,51332.74,2062.857647,44.482115,51550.325652,61.485844,80.485459,36.722699,False
65266,2021-05-13 07:00:00,51332.74,51367.19,50943.64,50947.68,1827.847841,41.529976,51398.816522,66.900729,82.315344,36.476152,False
65267,2021-05-13 07:30:00,50947.67,51000.00,50605.38,50879.72,1804.209253,41.012633,51262.160435,72.477554,83.837218,36.256573,False
65268,2021-05-13 08:00:00,50879.73,50950.00,50205.35,50476.23,2284.016803,37.987001,51090.686087,75.673123,76.774719,35.907044,False
65269,2021-05-13 08:30:00,50478.70,50674.99,49938.44,50015.82,2332.628022,34.829508,50886.293043,74.978484,58.255013,35.538514,False


In [41]:
df["rsi_sma_cross"] = qtpylib.crossed_above(df['rsi'], df['rsi_sma'])
# Toon alle crossings van RSI en SMA
df.loc[df['rsi_sma_cross'] == True].tail(10)

,date,open,high,low,close,volume,rsi,sma,slowd,slowk,rsi_sma,rsi_sma_cross
65037,2021-05-08 12:30:00,58539.82,59118.30,58431.81,58862.79,1711.744609,61.751101,58332.208696,63.814369,60.422320,59.867070,True
65048,2021-05-08 18:00:00,58901.45,59236.00,58901.45,59079.78,1868.935753,58.226317,58674.953043,43.358933,65.891621,57.579053,True
65052,2021-05-08 20:00:00,59030.56,59488.00,58755.00,59459.28,1250.270830,62.206334,58767.850435,73.582969,85.798161,57.131987,True
65092,2021-05-09 16:00:00,57379.58,57883.11,57300.01,57652.00,1638.513826,48.272958,57742.773913,41.410113,54.824907,46.905116,True
65100,2021-05-09 20:00:00,57266.97,57999.00,57179.18,57636.87,1672.375584,50.564507,57458.160870,63.107373,62.637096,45.724537,True
65139,2021-05-10 15:30:00,57366.89,58320.00,57200.00,58296.36,2823.245035,54.254951,58134.221304,19.955953,31.307560,50.958834,True
65164,2021-05-11 04:00:00,55142.44,55545.69,54986.47,55507.12,1505.356743,44.808249,55803.434783,50.711119,38.151121,42.923922,True
65168,2021-05-11 06:00:00,54788.60,55247.21,54721.89,55150.01,1142.895032,43.101232,55478.247391,39.819972,35.174511,40.956723,True
65218,2021-05-12 07:00:00,57011.67,57250.00,56990.28,57205.20,990.139085,55.463476,57014.671304,63.751312,42.013201,54.938693,True
65262,2021-05-13 05:00:00,50445.87,51076.41,50444.00,51051.72,1922.728800,40.785695,51963.655217,50.747652,57.601310,37.769987,True


In [43]:
df['stoch_sell_cross']=(df['slowd']>75)&(df['slowk']>75)&(qtpylib.crossed_above(df['slowd'], df['slowk']))
# Toon alle sell crossings
df.loc[df['stoch_sell_cross'] == True].tail(10)


,date,open,high,low,close,volume,rsi,sma,slowd,slowk,rsi_sma,rsi_sma_cross,stoch_sell_cross
64899,2021-05-05 15:30:00,57025.33,57712.06,56981.39,57636.74,3421.417182,73.169580,55431.119130,87.850280,87.718381,48.189296,False,True
64901,2021-05-05 16:30:00,57567.46,57718.97,57227.45,57414.77,2121.296876,69.453283,55683.968696,88.585966,85.313272,50.063158,False,True
64990,2021-05-07 13:00:00,57459.27,57680.00,57116.16,57160.38,1698.042215,62.843608,56150.929565,82.025643,79.098665,46.413879,False,True
65001,2021-05-07 18:30:00,57990.94,58119.77,57815.68,57890.03,1024.293973,64.163608,57010.404348,83.762109,76.457930,50.340271,False,True
65054,2021-05-08 21:00:00,58935.31,58992.55,58623.21,58830.21,943.749880,52.068911,58762.060435,80.784827,79.228565,56.898491,False,True
65108,2021-05-10 00:00:00,58240.83,58377.67,58009.96,58133.89,1304.753150,57.143626,57597.586087,85.085781,83.407459,46.313211,False,True
65111,2021-05-10 01:30:00,58683.04,58985.00,58610.16,58700.21,977.654541,63.772647,57807.600435,86.738177,86.656755,47.137887,False,True
65117,2021-05-10 04:30:00,59165.17,59359.99,59077.29,59154.67,1049.349293,66.288265,58246.065652,90.271972,88.507805,49.231656,False,True
65198,2021-05-11 21:00:00,56806.35,56862.43,56570.00,56576.70,667.930865,56.279471,56058.809565,91.936290,89.248552,46.944831,False,True
65213,2021-05-12 04:30:00,57680.79,57699.17,57284.82,57355.30,1065.213645,59.389167,56878.130870,84.709733,82.810540,53.606094,False,True


In [56]:
# Idee is om te controleren of bij het Moving Average 
# De huidige waarde > waarde-1
# & waarde-1 > waarde-2
# & waarde-2 > waarde-3

df['growing_sma']=(df['sma']>df['sma'].shift(1))&(df['sma'].shift(1)>df['sma'].shift(2))&(df['sma'].shift(2)>df['sma'].shift(3))
df[['date','close','sma','growing_sma']].loc[df['growing_sma'] == True].tail(50)

,date,close,sma,growing_sma
65116,2021-05-10 04:00:00,59165.16,58170.055652,True
65117,2021-05-10 04:30:00,59154.67,58246.065652,True
65118,2021-05-10 05:00:00,58919.98,58309.598261,True
65119,2021-05-10 05:30:00,59068.05,58378.643478,True
65120,2021-05-10 06:00:00,58930.99,58443.495652,True
65121,2021-05-10 06:30:00,58833.93,58517.439565,True
65122,2021-05-10 07:00:00,58819.65,58584.947391,True
65123,2021-05-10 07:30:00,58370.17,58616.830000,True
65124,2021-05-10 08:00:00,58271.14,58635.393478,True
65125,2021-05-10 08:30:00,58372.87,58659.138696,True


dataframe['last_lowest'] = dataframe['low'].rolling(10).min().shift(1)
dataframe['lower_low'] = dataframe['close'] < dataframe['last_lowest']
print(dataframe[['date','close','low','last_lowest','lower_low']].loc[dataframe['lower_low'] == True].tail(55))


In [58]:
df['ohlc']=(df['open']+df['high']+df['low']+df['close'])/4

df[['date','close','sma','ohlc']].tail(25)

,date,close,sma,ohlc
65245,2021-05-12 20:30:00,54420.14,55509.394783,54273.0900
65246,2021-05-12 21:00:00,54716.86,55416.413478,54547.2300
65247,2021-05-12 21:30:00,54533.16,55335.770000,54577.7150
65248,2021-05-12 22:00:00,52748.44,55193.231304,53507.1050
65249,2021-05-12 22:30:00,52942.79,55052.715217,52559.9425
65250,2021-05-12 23:00:00,51591.42,54842.713478,52189.4850
65251,2021-05-12 23:30:00,49631.32,54539.529130,50479.0900
65252,2021-05-13 00:00:00,49724.77,54252.920870,48876.4800
65253,2021-05-13 00:30:00,49666.99,53982.985217,49896.9400
65254,2021-05-13 01:00:00,50324.12,53703.159565,49972.6675


In [80]:
# Declining slowk no entry signal
df['slowk_decline']= (df['slowk'] > 70) & (df['slowk'] < df['slowk'].shift(1)) & (df['slowk'].shift(1) < df['slowk'].shift(2))
df[['date','slowk','slowk_decline']].loc[df.slowk_decline==True].tail(55)

,date,slowk,slowk_decline
64645,2021-04-30 08:30:00,85.120594,True
64648,2021-04-30 10:00:00,75.980968,True
64660,2021-04-30 16:00:00,90.709734,True
64661,2021-04-30 16:30:00,89.570551,True
64662,2021-04-30 17:00:00,82.772461,True
64663,2021-04-30 17:30:00,77.095675,True
64664,2021-04-30 18:00:00,75.293370,True
64669,2021-04-30 20:30:00,77.758894,True
64681,2021-05-01 02:30:00,90.006426,True
64684,2021-05-01 04:00:00,82.767450,True


In [82]:
df['ao'] = qtpylib.awesome_oscillator(df)
df[['date','close','ao']]

,date,close,ao
0,2017-08-17 04:00:00,4261.45,NaN
1,2017-08-17 04:30:00,4308.83,NaN
2,2017-08-17 05:00:00,4320.00,NaN
3,2017-08-17 05:30:00,4315.32,NaN
4,2017-08-17 06:00:00,4311.02,NaN
...,...,...,...
65265,2021-05-13 06:30:00,51332.74,-1798.156912
65266,2021-05-13 07:00:00,50947.68,-1524.216824
65267,2021-05-13 07:30:00,50879.72,-1350.902029
65268,2021-05-13 08:00:00,50476.23,-1268.968618
